In [22]:
import pandas
import matplotlib
import subprocess
import attr

In [34]:
import re

# измерить пустой хэндлер

In [65]:
@attr.s
class BenchmarkResult:
    avg_latency = attr.ib(default=.0)
    rps = attr.ib(default=.0)
    # output = attr.ib(default='')
    
    def fill(self, output):
        self.avg_latency = float(re.findall(r'Latency\s+(\d*\.\d*)ms', output)[0])
        self.rps = float(re.findall(r'Requests/sec:\s+(\d*\.\d*)', output)[0])
        

In [66]:
def run_benchmark(handler: str, rps: int, duration: int = 10):
    output = subprocess.getoutput(f'wrk -t1 -c10 -d{duration}s -R{rps} --u_latency {handler}')
    res = BenchmarkResult()
    res.fill(output)
    return res

In [67]:
r = run_benchmark('http://127.0.0.1:8890/_info', rps=10, duration=1)
r

BenchmarkResult(avg_latency=1.23, rps=10.96)

In [68]:
r = run_benchmark('http://127.0.0.1:8890/_info', rps=1000, duration=1)
r

BenchmarkResult(avg_latency=2.26, rps=979.4)

In [14]:
!wrk -t1 -c10 -d10s -R100 --u_latency http://127.0.0.1:8890/_info

Running 10s test @ http://127.0.0.1:8890/_info
  1 threads and 10 connections
  Thread Stats   Avg      Stdev     Max   +/- Stdev
    Latency     2.85ms    1.70ms  11.64ms   77.74%
    Req/Sec       -nan      -nan   0.00      0.00%
  Latency Distribution (HdrHistogram - Recorded Latency)
 50.000%    2.57ms
 75.000%    3.46ms
 90.000%    4.81ms
 99.000%    9.14ms
 99.900%   10.83ms
 99.990%   11.57ms
 99.999%   11.65ms
100.000%   11.65ms

  Detailed Percentile spectrum:
       Value   Percentile   TotalCount 1/(1-Percentile)

       0.341     0.000000            1         1.00
       1.097     0.100000         1997         1.11
       1.497     0.200000         3992         1.25
       1.869     0.300000         5989         1.43
       2.233     0.400000         7988         1.67
       2.569     0.500000         9982         2.00
       2.727     0.550000        10976         2.22
       2.893     0.600000        11982         2.50
       3.059     0.650000        12979         2.86
 

In [13]:
!wrk -t1 -c10 -d20s -R100 --u_latency http://127.0.0.1:8890/sleep

Running 20s test @ http://127.0.0.1:8890/sleep
  1 threads and 10 connections
  Thread calibration: mean lat.: 204.889ms, rate sampling interval: 610ms
  Thread Stats   Avg      Stdev     Max   +/- Stdev
    Latency   452.79ms   73.40ms 598.53ms   58.27%
    Req/Sec    98.00      0.00    98.00    100.00%
  Latency Distribution (HdrHistogram - Recorded Latency)
 50.000%  449.79ms
 75.000%  515.58ms
 90.000%  555.52ms
 99.000%  587.78ms
 99.900%  595.97ms
 99.990%  599.04ms
 99.999%  599.04ms
100.000%  599.04ms

  Detailed Percentile spectrum:
       Value   Percentile   TotalCount 1/(1-Percentile)

     318.719     0.000000            1         1.00
     354.047     0.100000           98         1.11
     377.599     0.200000          195         1.25
     401.663     0.300000          292         1.43
     425.983     0.400000          390         1.67
     449.791     0.500000          488         2.00
     463.615     0.550000          537         2.22
     475.647     0.600000      

## 